# Transcribe Recordings
paid service [Goodtape](http://goodtape.io) will be used for the transcriptions

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import requests
import pandas as pd
import os
import shutil
import io

# audio
from pydub import AudioSegment

# local imports
from lib_henryk.logger import *
from lib_henryk.config import *

## Params

In [3]:
# load api keys
from dotenv import load_dotenv
_ = load_dotenv()

# goodtape api key
api_key = os.getenv("API_KEY_GOODTAPE")

## Load Dataset

In [4]:
df = pd.read_csv(FILE_AUDIO_STATS_CSV)

## Run Transcriptions

In [5]:
# load just one record, could be last file
index = -3
file_path = DIR_WIADOMOSCI_DO_HENRYCZKA + '/' + df.iloc[index]['path']
file_name = df.iloc[index]['file'].replace(' ', '_')
callback_url = 'https://webhook.site/d3c707b2-c8cb-44cf-97a7-a27e9cfda38c'

In [6]:
# create buffer
buffer = io.BytesIO()

# load audio segment
recording = AudioSegment.from_file(file_path, format='m4a')

# truncate the recording to x%
truncate_ratio = 0.1
recording_truncated = recording[0:int(len(recording) * truncate_ratio)]

# write converted file to the buffer
recording_truncated.export(buffer, format='mp3')
recording_name = file_name.replace('.m4a', '.mp3')

In [7]:
url = "https://api.goodtape.io/transcribe"
# url = callback_url
headers = {"Authorization": api_key}
files = {"audio": (recording_name, buffer)}
data = {
    "callbackUrl": callback_url,
    "languageCode": "pl",
}

print(f'processing files: {files}')
response = requests.post(url, headers=headers, files=files, data=data)
print(response.json())

processing files: {'audio': ('Henryk_2024-04-29_Tata_opowiada_o_konkursie_i_nagrodach_a_Gastornis_uciekł_Tyranozaurowi.mp3', <_io.BytesIO object at 0x7f53189439c0>)}
{'detail': "Error processing request. You'll not be billed for this request."}
